In [ ]:
#Was currently on RMG-Database commit: 79d860c0654b2f55c0c9b3a13c066d0d02b71c75 (successfully autogenerated Intra_R_Add_Endocyclic family) #after initial autogeneration of Intra_R_endocyclic
#For debugging, git checkout the rules.py and groups.py in Intra_R_Endocyclic in this commit: b3a3a900e933b01e231ca40202c41cefc30b3947 #

In [1]:
import rmgpy
import numpy as np
from rmgpy.molecule.molecule import *
from rmgpy.species import *
from rmgpy.chemkin import *
from rmgpy.data.rmg import RMGDatabase
from IPython.display import display
from rmgpy.data.thermo import ThermoLibrary
from rmgpy.rmg.react import react
from rmgpy.species import Species
from rmgpy.reaction import Reaction
from rmgpy.data.rmg import get_db
from rmgpy.molecule.group import Group
from rmgpy.kinetics.arrhenius import ArrheniusBM
from rmgpy import settings
import time
import matplotlib.pyplot as plt
import matplotlib

In [2]:
settings

{'database.directory': '/home/khalil.nor/Code/RMG-database/input',
 'test_data.directory': '/home/khalil.nor/Code/RMG-Py/rmgpy/test_data'}

In [ ]:
#these families just had training rxns added to them 

#dict_keys(['1,2_Insertion_carbene', 'R_Addition_MultipleBond', 'XY_Addition_MultipleBond', 'Singlet_Carbene_Intra_Disproportionation', '1,2_Insertion_CO', '1+2_Cycloaddition', '1,3_Insertion_CO2', '1,3_sigmatropic_rearrangement'])

In [3]:
thermo_libs = [
'C1_C2_Fluorine', #adding Siddha's as first most trusted because this is the thermo library that Franklin used
'NCSU_C2_C8_PFAS', #adding Westmoreland's thermo as the second most trusted
'primaryThermoLibrary',
'Fluorine',
'FFCM1(-)',
'halogens',
'CHOF_G4',
'CHOCl_G4',
'CHOBr_G4',
'CHOFCl_G4',
'CHOFBr_G4',
'CHOFClBr_G4',
'DFT_QCI_thermo',
'2-BTP_G4',
'thermo_DFT_CCSDTF12_BAC',
'SulfurHaynes'
]

kin_families = ['R_Recombination', '1,2_Insertion_carbene', 'F_Abstraction', 'R_Addition_MultipleBond', 'XY_Addition_MultipleBond', 'Singlet_Carbene_Intra_Disproportionation', 'Intra_R_Add_Endocyclic', 'R_Addition_COm', '1,2_Insertion_CO', '1+2_Cycloaddition', '1,3_Insertion_CO2', '1,3_sigmatropic_rearrangement']


In [4]:
database = RMGDatabase()
database.load(
            path = settings['database.directory'],
            thermo_libraries = thermo_libs,
            transport_libraries = [],
            reaction_libraries = [],
            seed_mechanisms = [],#['BurkeH2O2inN2','ERC-FoundationFuelv0.9'],
            kinetics_families = kin_families,
            kinetics_depositories = ['training'],
            #frequenciesLibraries = self.statmechLibraries,
            depository = False, # Don't bother loading the depository information, as we don't use it
        )


1 *1 R u0 {2,[D,T]}
2 *2 R u0 {1,[D,T]}
3 *3 R u0 p[1,2,3]

1 *2 [H,Cs,Cd,Cb,Ct,CO,O,Sis,Sid,N,S]              u0 {2,S}
2 *3 [H,F1s,Cl1s,Br1s,I1s,Cs,Cd,Cb,Ct,Sis,Sid,N,S] u0 {1,S}
3 *1 C                                             u0 p1 c-1 {4,T}
4 *4 [O,S]                                         u0 p1 c+1 {3,T}

1 *1 C2s                                             u0 p1 {2,S} {3,S}
2    [H,F1s,Cl1s,Br1s,I1s]                           u0 {1,S}
3    [H,F1s,Cl1s,Br1s,I1s,C]                         u0 {1,S}
4 *2 [H,F1s,Cl1s,Br1s,I1s,Cs,Cd,Cb,Ct,O,Sis,Sid,N,S] u0 {5,S}
5 *3 [H,F1s,Cl1s,Br1s,I1s,Cs,Cd,Cb,Ct,Sis,Sid,N,S]   u0 {4,S}

1 *1 Cdd       u0 {2,D} {3,D}
2 *2 [O2d,S2d] u0 {1,D}
3    [O2d,S2d] u0 {1,D}
4 *3 R         u0 {5,S}
5 *4 R         u0 {4,S}

1 *2 R!H u0 {2,[S,D,T,B]} {3,[D,T,B]}
2 *1 R!H u1 {1,[S,D,T,B]}
3 *3 R!H u0 c0 {1,[D,T,B]}

1 *4 R!H ux {2,[S,D,T,B]} {3,[S,D,T,B]}
2 *2 R!H u0 {1,[S,D,T,B]} {4,[D,T,B]}
3 *1 R!H u1 {1,[S,D,T,B]}
4 *3 R!H u0 c0 {2,[D,T,B]}

1 *5 R!H ux {2

In [ ]:
database.kinetics.families

In [ ]:
# 'Intra_R_Add_Endocyclic' 
family_to_train = "Intra_R_Add_Endocyclic"
family = database.kinetics.families[family_to_train]

In [ ]:
family.clean_tree()

In [ ]:
start = time.time()
family.generate_tree(thermo_database=database.thermo,
                     nprocs=1,
                     new_fraction_threshold_to_reopt_node=0.25,
                     max_batch_size=800,
                     extension_iter_max=2,
                     extension_iter_item_cap=100)

end = time.time()
print(end-start)

In [ ]:
len(family.groups.entries)

In [ ]:
family.groups.entries

In [ ]:
start = time.time()
family.check_tree()
end = time.time()
print(end-start)

In [ ]:
start = time.time()
family.regularize(thermo_database=database.thermo)
end = time.time()
print(end-start)

In [ ]:
start = time.time()
templateRxnMap = family.get_reaction_matches(thermo_database=database.thermo,remove_degeneracy=True,
                                             get_reverse=True,exact_matches_only=False,fix_labels=True)
end = time.time()
print(end-start)

In [ ]:
len(templateRxnMap)

In [ ]:
family.clean_tree_rules()

In [ ]:
start = time.time()
family.make_bm_rules_from_template_rxn_map(templateRxnMap)#,nprocs=6)
end = time.time()
print(end-start)

In [ ]:
start = time.time()
family.check_tree()
end = time.time()
print(end-start)

In [ ]:
start = time.time()
errors,uncertainties = family.cross_validate(iters=0,random_state=5,folds=0,ascend=False)
end = time.time()
print(end-start)

In [ ]:
save_path = os.path.join(settings['database.directory'], 'kinetics', 'families', family.name)
print(save_path)

In [ ]:
family.save(save_path)

In [ ]:
#completed on May 14th, 2024 at 11:53 am